In [1]:
import arcpy
import os

# 定义坐标 (左下角、右下角、右上角、左上角)
coordinates = [
 [97, 25.5],   
 [109, 25.5],  
  [109, 34.5], [97, 34.5],  [97, 25.5]
  
  
]

# 创建空间参考（例如：WGS 1984）
spatial_ref = arcpy.SpatialReference(4326)  # WGS 1984

# 创建多边形，并指定空间参考
polygon = arcpy.Polygon(arcpy.Array([arcpy.Point(*coord) for coord in coordinates]), spatial_ref)

# 获取当前路径
current_path = r"C:\Users\r\Desktop\arcen"

# 指定 sichuan_loc 文件夹路径
loc_folder = os.path.join(current_path, "arcen", "sichuan_loc")

# 如果 sichuan_loc 文件夹不存在，创建它
if not os.path.exists(loc_folder):
    os.makedirs(loc_folder)

# 创建一个新的要素类并保存多边形到 sichuan_loc 文件夹
gdb_path = os.path.join(loc_folder, "sichuan_loc.gdb")
arcpy.management.CreateFileGDB(loc_folder, "sichuan_loc")  # 创建 sichuan_loc.gdb
arcpy.management.CreateFeatureclass(gdb_path, "sichuan_loc", "POLYGON", spatial_reference=spatial_ref)  # 创建 sichuan_loc 要素类，并指定空间参考

# 插入多边形到要素类
with arcpy.da.InsertCursor(os.path.join(gdb_path, "sichuan_loc"), ["SHAPE@"]) as cursor:
    cursor.insertRow([polygon])

# 设置符号系统
aprx_path = r"C:\Users\r\Desktop\arcen\arcen\arcen.aprx"  # 指定 .aprx 文件的路径
aprx = arcpy.mp.ArcGISProject(aprx_path)  # 加载指定的项目文件
map_obj = aprx.listMaps()[0]  # 获取第一个地图
lyr = map_obj.addDataFromPath(os.path.join(gdb_path, "sichuan_loc"))  # 添加数据到地图
sym = lyr.symbology  # 获取符号系统
sym.renderer.symbol.color = {'RGB': [255, 0, 0, 100]}  # 设置红色
lyr.symbology = sym  # 应用符号系统

# Optional: Save the project if you want to keep the changes
aprx.save()